In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, Ridge, Lasso

import optuna

from common import EP
from dfdb import DFDB

import types
import copy

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [7]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [8]:
catboost_columns = ['spkt_welch_density__coeff_3',
 'spkt_welch_densitycoeff_2',
 'abs_q25_5',
 'abs_q75_6',
 'q05_roll_std_1000',
 'abs_q75_7',
 'abs_q95_2',
 'q05_5',
 'abs_q75_2',
 '5000skewness_max_',
 'fft_coefficientcoeff_80__attr_"imag"',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 '5000kurtosis_mean_',
 "number_peaks{'n': 1}",
 '5000smoothness_entropy_',
 'ave10_7',
 'q75_roll_std_1000',
 'FFT_Mag_25q0',
 'fft_coefficientcoeff_20__attr_"abs"']
lgbm_columns = ['q25_roll_std_100',
 'abs_q25_5',
 'spkt_welch_density__coeff_3',
 'abs_q75_6',
 'abs_q75_7',
 'spkt_welch_densitycoeff_2',
 'median__roll_std',
 'abs_q01_5',
 '5000smoothness_quantile05',
 '5000smoothness_std_',
 'abs_q95_3',
 'FFT_Mag_75q0',
 '5000median_std_',
 'spkt_welch_density__coeff_17']
xgbm_columns = ['q25_roll_std_100',
 'abs_q25_5',
 'spkt_welch_density__coeff_3',
 'abs_q75_7',
 'spkt_welch_densitycoeff_2',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'agg_autocorrelationf_agg_"mean"__maxlag_40',
 "number_peaks{'n': 1}",
 '5000peak_peak_amp_max_',
 'abs_q95_3',
 'spkt_welch_density__coeff_89',
 'abs_q05_2']
randomforest_randomforest = ['abs_q25_5', 'abs_q01_4', 'q25_roll_std_100']
extratrees_columns = ['q05_2',
 "number_peaks{'n': 1}",
 'abs_q01_6',
 'abs_q95_2',
 '5000smoothness_quantile25',
 '5000std_median_',
 '5000smoothness_median_',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'abs_q75_6',
 "number_peaks{'n': 3}",
 'q75_roll_std_10',
 "number_peaks{'n': 10}",
 '5000min_quantile75',
 '5000smoothness_quantile05',
 "number_peaks{'n': 5}",
 'abs_q01_2',
 '5000smoothness_mean_',
 'min_roll_std_100',
 'abs_q05_2',
 'q01_roll_std_1000']
gradientboosting_columns = ['q05_5',
 'kurt_1',
 'abs_q75_6',
 'abs_q75_7',
 'spkt_welch_density__coeff_28',
 'spkt_welch_density__coeff_99',
 'fft_coefficientcoeff_6__attr_"abs"',
 '5000smoothness_quantile05',
 'q25_roll_std_100',
 'spkt_welch_densitycoeff_2',
 'abs_max_1',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 'abs_q25_5',
 'abs_q01_7',
 'q05_8',
 'Hilbert_mean_6',
 'abs_q95_2',
 '5000skewness_max_',
 '5000kurtosis_mean_',
 'spkt_welch_density__coeff_3']

In [9]:
all_columns = catboost_columns+lgbm_columns+xgbm_columns+randomforest_randomforest+extratrees_columns+gradientboosting_columns
unique_columns = list(set(all_columns))
common_columns = []
common_columns50 = []
common_columns75 = []
common_columns95 = []
N_columns = 6
count_values = [all_columns.count(col) for col in unique_columns]
for col in unique_columns:
    if all_columns.count(col)==N_columns:
        common_columns.append(col)
    if all_columns.count(col)>=np.quantile(count_values, .5):
        common_columns50.append(col)
    if all_columns.count(col)>=np.quantile(count_values, .75):
        common_columns75.append(col)
    if all_columns.count(col)>=np.quantile(count_values, .95):
        common_columns95.append(col)
print('unique_columns ',len(unique_columns))
print('common_columns50 ',len(common_columns50))
print('common_columns75 ',len(common_columns75))
print('common_columns95 ',len(common_columns95))
print('common_columns ',len(common_columns))

unique_columns  55
common_columns50  55
common_columns75  15
common_columns95  7
common_columns  0


In [10]:
db = DFDB('../trial/ridge.pkl', auto_commit=False)

In [16]:
param={'algorithm': {'cls': 'Ridge',
  'fit': {},
  'init': {'alpha': 50000,
   'fit_intercept': True,
   'normalize': False,
   'copy_X': True,
   'max_iter': None,
   'tol': 0.001,
   'solver': 'auto',
   'random_state': 42}},
 'columns': tsfresh_columns,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},#stratified
 'scaler': {'cls': 'StandardScaler', 'init':{}}}

mytrial = []
columns = common_columns50

# param['algorithm']['init'] = {'logging_level': 'Silent', **param['algorithm']['init']}
# param['scaler']['init'] = {}
# param['algorithm']['fit'] = {}

selected_columns = EP.revert_rfe(df_train, param, columns, df_test, mytrial, start_columns=common_columns95, limit=70, remark='start from top1 column2')
print(len(selected_columns))
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])

44


In [17]:
df_trial[(df_trial['remark']=='start from top1 column2')][['datetime','nfeatures','kfold', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,kfold,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
144,2019-05-25 06:00:19.594532,7,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",2.427872,0.002716,2.426909,0.012184,-0.000964,start from top1 column2
145,2019-05-25 06:00:19.766483,8,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",2.418556,0.002892,2.417939,0.012465,-0.000618,start from top1 column2
146,2019-05-25 06:00:20.001372,9,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",2.336255,0.003221,2.336020,0.013136,-0.000235,start from top1 column2
147,2019-05-25 06:00:20.154962,9,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",2.336255,0.003221,2.336020,0.013136,-0.000235,start from top1 column2
148,2019-05-25 06:00:20.270794,10,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",2.333407,0.003311,2.333299,0.013080,-0.000108,start from top1 column2
149,2019-05-25 06:00:20.416773,11,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",2.314546,0.003207,2.315087,0.012646,0.000542,start from top1 column2
150,2019-05-25 06:00:20.574299,11,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",2.314546,0.003207,2.315087,0.012646,0.000542,start from top1 column2
151,2019-05-25 06:00:20.715075,12,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",2.314013,0.003199,2.314635,0.012554,0.000622,start from top1 column2
152,2019-05-25 06:00:20.880037,13,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",2.311676,0.003207,2.312251,0.012716,0.000575,start from top1 column2
153,2019-05-25 06:00:20.982352,14,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",2.293438,0.003232,2.294248,0.012660,0.000810,start from top1 column2


In [19]:
mytrial = []
#  tune hypterparameters
def objective(trial):

    alpha = trial.suggest_int('alpha', 10000, 100000)
    tol = trial.suggest_uniform('tol', 0.0001, 0.01)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={'algorithm': {'cls': 'Ridge',
      'fit': {},
      'init': {'alpha': alpha,
       'fit_intercept': True,
       'normalize': False,
       'copy_X': True,
       'max_iter': None,
       'tol': tol,
       'solver': 'auto',
       'random_state': random_state}},
     'columns': selected_columns,
     'kfold': {'n_splits': 8,
      'random_state': 1985,
      'shuffle': True,
      'type': 'group'},#stratified
     'scaler': {'cls': 'StandardScaler', 'init':{}}}
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 199')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-25 06:11:45,446] Finished a trial resulted in value: 0.08453327793966746. Current best value is 0.08453327793966746 with parameters: {'alpha': 98239, 'tol': 0.004737167962107372, 'random_state': 8565}.
[I 2019-05-25 06:11:46,878] Finished a trial resulted in value: 0.10673501466942746. Current best value is 0.08453327793966746 with parameters: {'alpha': 98239, 'tol': 0.004737167962107372, 'random_state': 8565}.
[I 2019-05-25 06:11:48,237] Finished a trial resulted in value: 0.10815308421231846. Current best value is 0.08453327793966746 with parameters: {'alpha': 98239, 'tol': 0.004737167962107372, 'random_state': 8565}.
[I 2019-05-25 06:11:49,757] Finished a trial resulted in value: 0.09143753339355495. Current best value is 0.08453327793966746 with parameters: {'alpha': 98239, 'tol': 0.004737167962107372, 'random_state': 8565}.
[I 2019-05-25 06:11:51,508] Finished a trial resulted in value: 0.09038533143756433. Current best value is 0.08453327793966746 with parameters: {'al

[I 2019-05-25 06:12:40,798] Finished a trial resulted in value: 0.09653383345211416. Current best value is 0.08430308298522199 with parameters: {'alpha': 99943, 'tol': 0.004922169259521192, 'random_state': 5571}.
[I 2019-05-25 06:12:42,294] Finished a trial resulted in value: 0.08643913784523288. Current best value is 0.08430308298522199 with parameters: {'alpha': 99943, 'tol': 0.004922169259521192, 'random_state': 5571}.
[I 2019-05-25 06:12:43,587] Finished a trial resulted in value: 0.0851445370877008. Current best value is 0.08430308298522199 with parameters: {'alpha': 99943, 'tol': 0.004922169259521192, 'random_state': 5571}.
[I 2019-05-25 06:12:44,849] Finished a trial resulted in value: 0.08729375874344138. Current best value is 0.08430308298522199 with parameters: {'alpha': 99943, 'tol': 0.004922169259521192, 'random_state': 5571}.
[I 2019-05-25 06:12:46,544] Finished a trial resulted in value: 0.10654494416007827. Current best value is 0.08430308298522199 with parameters: {'alp

[I 2019-05-25 06:13:35,816] Finished a trial resulted in value: 0.08662848821929632. Current best value is 0.08430308298522199 with parameters: {'alpha': 99943, 'tol': 0.004922169259521192, 'random_state': 5571}.
[I 2019-05-25 06:13:36,945] Finished a trial resulted in value: 0.08429912790331356. Current best value is 0.08429912790331356 with parameters: {'alpha': 99972, 'tol': 0.006469831509060518, 'random_state': 4566}.
[I 2019-05-25 06:13:38,462] Finished a trial resulted in value: 0.08721404949628606. Current best value is 0.08429912790331356 with parameters: {'alpha': 99972, 'tol': 0.006469831509060518, 'random_state': 4566}.
[I 2019-05-25 06:13:39,489] Finished a trial resulted in value: 0.09852865654666758. Current best value is 0.08429912790331356 with parameters: {'alpha': 99972, 'tol': 0.006469831509060518, 'random_state': 4566}.
[I 2019-05-25 06:13:40,734] Finished a trial resulted in value: 0.08536584936815149. Current best value is 0.08429912790331356 with parameters: {'al

[I 2019-05-25 06:14:23,223] Finished a trial resulted in value: 0.08970198372090787. Current best value is 0.08429912790331356 with parameters: {'alpha': 99972, 'tol': 0.006469831509060518, 'random_state': 4566}.
[I 2019-05-25 06:14:24,301] Finished a trial resulted in value: 0.08698405487882961. Current best value is 0.08429912790331356 with parameters: {'alpha': 99972, 'tol': 0.006469831509060518, 'random_state': 4566}.
[I 2019-05-25 06:14:25,245] Finished a trial resulted in value: 0.08850016952842346. Current best value is 0.08429912790331356 with parameters: {'alpha': 99972, 'tol': 0.006469831509060518, 'random_state': 4566}.
[I 2019-05-25 06:14:26,272] Finished a trial resulted in value: 0.08562031802202066. Current best value is 0.08429912790331356 with parameters: {'alpha': 99972, 'tol': 0.006469831509060518, 'random_state': 4566}.
[I 2019-05-25 06:14:27,312] Finished a trial resulted in value: 0.1055920514783602. Current best value is 0.08429912790331356 with parameters: {'alp

[I 2019-05-25 06:15:07,787] Finished a trial resulted in value: 0.08540851667090377. Current best value is 0.08429912790331356 with parameters: {'alpha': 99972, 'tol': 0.006469831509060518, 'random_state': 4566}.
[I 2019-05-25 06:15:08,773] Finished a trial resulted in value: 0.08656230401609104. Current best value is 0.08429912790331356 with parameters: {'alpha': 99972, 'tol': 0.006469831509060518, 'random_state': 4566}.
[I 2019-05-25 06:15:10,053] Finished a trial resulted in value: 0.08430539981552956. Current best value is 0.08429912790331356 with parameters: {'alpha': 99972, 'tol': 0.006469831509060518, 'random_state': 4566}.
[I 2019-05-25 06:15:11,275] Finished a trial resulted in value: 0.08455926967863871. Current best value is 0.08429912790331356 with parameters: {'alpha': 99972, 'tol': 0.006469831509060518, 'random_state': 4566}.
[I 2019-05-25 06:15:12,510] Finished a trial resulted in value: 0.08513103609153858. Current best value is 0.08429912790331356 with parameters: {'al

[I 2019-05-25 06:15:56,680] Finished a trial resulted in value: 0.08697613245173161. Current best value is 0.08429598978358455 with parameters: {'alpha': 99995, 'tol': 0.003534252670347287, 'random_state': 7719}.
[I 2019-05-25 06:15:58,413] Finished a trial resulted in value: 0.10880290870546323. Current best value is 0.08429598978358455 with parameters: {'alpha': 99995, 'tol': 0.003534252670347287, 'random_state': 7719}.
[I 2019-05-25 06:15:59,626] Finished a trial resulted in value: 0.08553761328089882. Current best value is 0.08429598978358455 with parameters: {'alpha': 99995, 'tol': 0.003534252670347287, 'random_state': 7719}.
[I 2019-05-25 06:16:00,965] Finished a trial resulted in value: 0.08653701909173556. Current best value is 0.08429598978358455 with parameters: {'alpha': 99995, 'tol': 0.003534252670347287, 'random_state': 7719}.
[I 2019-05-25 06:16:02,554] Finished a trial resulted in value: 0.08498442102222527. Current best value is 0.08429598978358455 with parameters: {'al

In [21]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [22]:
df_trial[(df_trial['remark']=='tune 199')&(df_trial['mae_diff']<.1)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
251,2019-05-25 06:12:57.901092,tune 199,44,2.116238,0.013467,2.167211,0.533201,0.050973
391,2019-05-25 06:15:58.381449,tune 199,44,2.119027,0.013523,2.169186,0.534659,0.050158
202,2019-05-25 06:11:48.235489,tune 199,44,2.120168,0.013546,2.170008,0.535256,0.049840
291,2019-05-25 06:13:56.203938,tune 199,44,2.120498,0.013553,2.170247,0.535429,0.049749
201,2019-05-25 06:11:46.875805,tune 199,44,2.122797,0.013601,2.171940,0.536617,0.049143
242,2019-05-25 06:12:46.534242,tune 199,44,2.123161,0.013608,2.172210,0.536801,0.049049
318,2019-05-25 06:14:27.291500,tune 199,44,2.124991,0.013647,2.173571,0.537726,0.048580
208,2019-05-25 06:11:57.346792,tune 199,44,2.127512,0.013701,2.175461,0.538968,0.047949
383,2019-05-25 06:15:46.987406,tune 199,44,2.128013,0.013712,2.175839,0.539210,0.047826
292,2019-05-25 06:13:57.345675,tune 199,44,2.129559,0.013745,2.177014,0.539940,0.047455


In [23]:
db.commit()

In [24]:
mytrial = []
#  tune hypterparameters
def objective(trial):

    alpha = trial.suggest_int('alpha', 10000, 100000)
    tol = trial.suggest_uniform('tol', 0.0001, 0.01)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={'algorithm': {'cls': 'Ridge',
      'fit': {},
      'init': {'alpha': alpha,
       'fit_intercept': True,
       'normalize': False,
       'copy_X': True,
       'max_iter': None,
       'tol': tol,
       'solver': 'auto',
       'random_state': random_state}},
     'columns': selected_columns,
     'kfold': {'n_splits': 3,
      'random_state': 1985,
      'shuffle': True,
      'type': 'stratified'},#stratified
     'scaler': {'cls': 'StandardScaler', 'init':{}}}
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 199 by stratified')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-25 06:17:25,094] Finished a trial resulted in value: 0.002721089680975574. Current best value is 0.002721089680975574 with parameters: {'alpha': 13520, 'tol': 0.0014058350786986808, 'random_state': 8560}.
[I 2019-05-25 06:17:25,626] Finished a trial resulted in value: 0.0012173685775940868. Current best value is 0.0012173685775940868 with parameters: {'alpha': 91705, 'tol': 0.0012359104667916173, 'random_state': 8816}.
[I 2019-05-25 06:17:25,991] Finished a trial resulted in value: 0.0017548573424844588. Current best value is 0.0012173685775940868 with parameters: {'alpha': 91705, 'tol': 0.0012359104667916173, 'random_state': 8816}.
[I 2019-05-25 06:17:26,379] Finished a trial resulted in value: 0.0016128793024456677. Current best value is 0.0012173685775940868 with parameters: {'alpha': 91705, 'tol': 0.0012359104667916173, 'random_state': 8816}.
[I 2019-05-25 06:17:26,893] Finished a trial resulted in value: 0.0013450228661690822. Current best value is 0.0012173685775940868

[I 2019-05-25 06:17:42,319] Finished a trial resulted in value: 0.0017694483541649973. Current best value is 0.0011335587417723378 with parameters: {'alpha': 99868, 'tol': 0.0030549056544553537, 'random_state': 3376}.
[I 2019-05-25 06:17:42,792] Finished a trial resulted in value: 0.0013330346431594667. Current best value is 0.0011335587417723378 with parameters: {'alpha': 99868, 'tol': 0.0030549056544553537, 'random_state': 3376}.
[I 2019-05-25 06:17:43,368] Finished a trial resulted in value: 0.001201795033893168. Current best value is 0.0011335587417723378 with parameters: {'alpha': 99868, 'tol': 0.0030549056544553537, 'random_state': 3376}.
[I 2019-05-25 06:17:43,967] Finished a trial resulted in value: 0.0013436965144458914. Current best value is 0.0011335587417723378 with parameters: {'alpha': 99868, 'tol': 0.0030549056544553537, 'random_state': 3376}.
[I 2019-05-25 06:17:44,529] Finished a trial resulted in value: 0.0015157816520844495. Current best value is 0.001133558741772337

[I 2019-05-25 06:18:01,862] Finished a trial resulted in value: 0.0011479806434083414. Current best value is 0.0011335587417723378 with parameters: {'alpha': 99868, 'tol': 0.0030549056544553537, 'random_state': 3376}.
[I 2019-05-25 06:18:02,285] Finished a trial resulted in value: 0.0014043814515606351. Current best value is 0.0011335587417723378 with parameters: {'alpha': 99868, 'tol': 0.0030549056544553537, 'random_state': 3376}.
[I 2019-05-25 06:18:02,840] Finished a trial resulted in value: 0.0013491113571946103. Current best value is 0.0011335587417723378 with parameters: {'alpha': 99868, 'tol': 0.0030549056544553537, 'random_state': 3376}.
[I 2019-05-25 06:18:03,358] Finished a trial resulted in value: 0.0013450777302209422. Current best value is 0.0011335587417723378 with parameters: {'alpha': 99868, 'tol': 0.0030549056544553537, 'random_state': 3376}.
[I 2019-05-25 06:18:03,961] Finished a trial resulted in value: 0.0011333478509196189. Current best value is 0.00113334785091961

[I 2019-05-25 06:18:20,823] Finished a trial resulted in value: 0.001237592773228701. Current best value is 0.0011332073036277226 with parameters: {'alpha': 99969, 'tol': 0.00949967764321448, 'random_state': 3399}.
[I 2019-05-25 06:18:21,213] Finished a trial resulted in value: 0.0022771015088104865. Current best value is 0.0011332073036277226 with parameters: {'alpha': 99969, 'tol': 0.00949967764321448, 'random_state': 3399}.
[I 2019-05-25 06:18:21,654] Finished a trial resulted in value: 0.001204425697562295. Current best value is 0.0011332073036277226 with parameters: {'alpha': 99969, 'tol': 0.00949967764321448, 'random_state': 3399}.
[I 2019-05-25 06:18:22,148] Finished a trial resulted in value: 0.00128899088531744. Current best value is 0.0011332073036277226 with parameters: {'alpha': 99969, 'tol': 0.00949967764321448, 'random_state': 3399}.
[I 2019-05-25 06:18:22,621] Finished a trial resulted in value: 0.0016164784403092855. Current best value is 0.0011332073036277226 with para

[I 2019-05-25 06:18:36,036] Finished a trial resulted in value: 0.0011988419880747173. Current best value is 0.0011330891655631629 with parameters: {'alpha': 99998, 'tol': 0.006209519752197025, 'random_state': 3783}.
[I 2019-05-25 06:18:36,592] Finished a trial resulted in value: 0.001133533669267498. Current best value is 0.0011330891655631629 with parameters: {'alpha': 99998, 'tol': 0.006209519752197025, 'random_state': 3783}.
[I 2019-05-25 06:18:37,102] Finished a trial resulted in value: 0.0013516019589021112. Current best value is 0.0011330891655631629 with parameters: {'alpha': 99998, 'tol': 0.006209519752197025, 'random_state': 3783}.
[I 2019-05-25 06:18:37,489] Finished a trial resulted in value: 0.001162471251954955. Current best value is 0.0011330891655631629 with parameters: {'alpha': 99998, 'tol': 0.006209519752197025, 'random_state': 3783}.
[I 2019-05-25 06:18:37,967] Finished a trial resulted in value: 0.0012446489843890105. Current best value is 0.0011330891655631629 wit

[I 2019-05-25 06:18:52,338] Finished a trial resulted in value: 0.0011584431026470214. Current best value is 0.0011330891655631629 with parameters: {'alpha': 99998, 'tol': 0.006209519752197025, 'random_state': 3783}.
[I 2019-05-25 06:18:52,779] Finished a trial resulted in value: 0.0012707377770551744. Current best value is 0.0011330891655631629 with parameters: {'alpha': 99998, 'tol': 0.006209519752197025, 'random_state': 3783}.
[I 2019-05-25 06:18:53,146] Finished a trial resulted in value: 0.0015000598658780346. Current best value is 0.0011330891655631629 with parameters: {'alpha': 99998, 'tol': 0.006209519752197025, 'random_state': 3783}.
[I 2019-05-25 06:18:53,556] Finished a trial resulted in value: 0.0012018919866764383. Current best value is 0.0011330891655631629 with parameters: {'alpha': 99998, 'tol': 0.006209519752197025, 'random_state': 3783}.
[I 2019-05-25 06:18:53,986] Finished a trial resulted in value: 0.0012344938441716457. Current best value is 0.0011330891655631629 w

In [25]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [27]:
df_trial[(df_trial['remark']=='tune 199 by stratified')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
429,2019-05-25 06:17:38.650938,tune 199 by stratified,44,2.125651,2.556749e-07,2.127102,0.000027,0.001451
488,2019-05-25 06:18:09.446548,tune 199 by stratified,44,2.128753,1.425265e-07,2.130095,0.000028,0.001342
540,2019-05-25 06:18:32.977733,tune 199 by stratified,44,2.130999,7.708216e-08,2.132285,0.000029,0.001285
400,2019-05-25 06:17:25.092309,tune 199 by stratified,44,2.131436,6.674981e-08,2.132712,0.000029,0.001276
590,2019-05-25 06:18:54.312100,tune 199 by stratified,44,2.134129,2.196701e-08,2.135354,0.000030,0.001225
436,2019-05-25 06:17:41.725778,tune 199 by stratified,44,2.136483,5.845948e-09,2.137659,0.000032,0.001175
506,2019-05-25 06:18:18.631664,tune 199 by stratified,44,2.136944,5.467880e-09,2.138109,0.000032,0.001166
409,2019-05-25 06:17:29.083208,tune 199 by stratified,44,2.138588,1.136065e-08,2.139723,0.000033,0.001135
570,2019-05-25 06:18:45.806423,tune 199 by stratified,44,2.142156,5.566719e-08,2.143227,0.000035,0.001071
464,2019-05-25 06:17:56.549375,tune 199 by stratified,44,2.142272,5.787892e-08,2.143340,0.000035,0.001069


In [28]:
db.commit()